In [1]:
import tifffile as tiff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from skimage.filters import threshold_otsu
from skimage.morphology import convex_hull_image

In [2]:
img = 'data/dapi_cd9.tif'

In [3]:
def load_image(img_file):
    with tiff.TiffFile(img_file) as tif:
        images = tif.asarray().astype(float)
    return(images)

In [4]:
images = load_image(img)

In [5]:
images[0]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [11]:
def contour_image(raw_images, ch1, ch2):
    (chan1, chan2) = ch1 - 1, ch2 - 1

    img_shape = raw_images[0].shape

    raw_img = raw_images[[chan1, chan2]]
    contour_img = raw_img.copy()

    thresh1 = threshold_otsu(contour_img[0])
    binary1 = contour_img[0] > thresh1
    chull1 = convex_hull_image(binary1)

    thresh2 = threshold_otsu(contour_img[1])
    binary2 = contour_img[1] > thresh2
    chull2 = convex_hull_image(binary2)

    if sum(chull1.flatten()) > sum(chull2.flatten()):
        contour_img[0][chull1 == False] = np.nan
        contour_img[1][chull1 == False] = np.nan

    else:
        contour_img[0][chull2 == False] = np.nan
        contour_img[1][chull2 == False] = np.nan

    d = {}
    for i, data in enumerate(raw_img):
        d["orig_chan{}".format(i + 1)] = raw_img[i].flatten()
        d["contour_chan{}".format(i + 1)] = contour_img[i].flatten()

    return(pd.DataFrame(d), img_shape)

In [12]:
data, img_shape= contour_image(images, 1, 3)

In [13]:
data.max()

contour_chan1    2167.0
contour_chan2    1969.0
orig_chan1       2167.0
orig_chan2       1969.0
dtype: float64

In [28]:
def MandersCoef(final_df, img_shape):
    A = final_df['orig_chan1'].values.reshape(img_shape)
    B = final_df['orig_chan2'].values.reshape(img_shape)
    
    tA = final_df['contour_chan1'].values.reshape(img_shape)
    tB = final_df['contour_chan2'].values.reshape(img_shape)
       
    MA = (np.nansum(tB * A) / np.nansum(A)) / 100
    MB = (np.nansum(tA * B) / np.nansum(B)) / 100
    
    return MA, MB

In [29]:
MA, MB = MandersCoef(data, img_shape)

In [31]:
MA

91.149180289839961

In [24]:
test = np.array([0, np.nan, 3, np.nan])

In [25]:
test.sum()

nan

In [32]:
d = {'a':[1,2,3,4,5], 'b':[5,4,3,2,1]}
df = pd.DataFrame(d)

In [34]:
df['b'] = [1,1,1,1,1]

In [35]:
df

,a,b
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [36]:
df['c'] = [6,6,6,6,6]

In [37]:
df

,a,b,c
0,1,1,6
1,2,1,6
2,3,1,6
3,4,1,6
4,5,1,6


In [38]:
df[['a', 'c']]

,a,c
0,1,6
1,2,6
2,3,6
3,4,6
4,5,6


In [40]:
type(df['a'])

pandas.core.series.Series

In [41]:
df

,a,b,c
0,1,1,6
1,2,1,6
2,3,1,6
3,4,1,6
4,5,1,6


In [42]:
df2 = pd.DataFrame({'b':[2,2,2,2,2], 'c':[3,3,3,3,3]})

In [43]:
df2

,b,c
0,2,3
1,2,3
2,2,3
3,2,3
4,2,3


In [44]:
df[['b','c']] = df2

In [45]:
df

,a,b,c
0,1,2,3
1,2,2,3
2,3,2,3
3,4,2,3
4,5,2,3


In [ ]:
t